# Interactuando con Twitter


## Introducción

En este cuaderno vamos a crear un [*bot* de twitter](http://en.wikipedia.org/wiki/Twitterbot) (un programa que interactúa de manera semi-automática con twitter) que muestre ciertas capacidades lingüísticas.

A continuación vamos a revisar algunos *snippets* o ejemplos de código mostrando las instrucciones mínimas que puede incluir un *bot* para que sea capaz de publicar mensajes. También trataré de mostrar cómo acceder a otras funcionalidades, algo más complejas.


## Cuenta de desarrollador en twitter

Para poder crear un *bot* y, en general, cualquier aplicación que interactúe con [twitter](http://www.twitter.com), necesitas tener una cuenta con permisos de desarrollador. Sigue los pasos:

1. Lo primero que tienes que hacer, si no tienes cuenta en twitter, es [registrarte](https://twitter.com/signup). Si ya tienes cuenta en esta red, te recomiendo que la uses y no te crees otra nueva. El proceso de alta de twitter requiere la validación de un número de teléfono móvil único para cada cuenta. Así que, si no tienes más de un número de móvil y no quieres dolores de cabeza, utiliza tu cuenta de todos los días.

2. Una vez registrado, haz login con tu cuenta y entra en [la página de gestión de *apps* de twitter](https://apps.twitter.com), donde podrás solicitar permisos para una *app* nueva. Pincha en [*Create New App*](https://apps.twitter.com/app/new).

3. Rellena los campos requeridos del formulario que se muestra: 

    - ***Name***: tiene que ser un nombre único, así que sé original. Es probable que tengas que probar varios nombres hasta dar con uno que esté libre. Si no quieres fallar, pon algo como *progplnbot-tunombre*.
        
    - ***Description***: describe brevemente el propósito de tu bot.
        
    - ***Website***: por el momento, cualquier URL servirá. Si no tienes web propia y no sabes qué poner, introduce la URL de la web de Lingüística: http://www.ucm.es/linguistica
        
4. Lee (*cough, cough*) el texto del acuerdo, selecciona *Yes, I agree* y pincha el botón de la parte inferior de la página que dice *Create your Twitter application*.

5. Si todo ha ido bien (si has elegido para tu *app* un nombre que estuviera libre, básicamente), se te mostrará una página de configuración. Tendrás que cambiar algunos parámetros. 

    - En el apartado *Application Settings*, en *Access level*, asegúrate de que tienes seleccionados permisos de lectura y escritura: *Read and Write*. Si no es así, pincha en *modify app permissions* y otorga permisos para leer y escribir. Una vez cambiado, pincha en *Update Settings*.
        
    - En la pestaña *Keys and Access Tokens*, pincha en el botón *Create my access token* para crear los tokens de acceso. Estos nombres probablemente no te digan nada, pero desde esta página tienes acceso a las cuatro credenciales que tu *bot* necesita para autenticarse y publicar mensajes. Toma nota de ellas, las necesitarás más adelante:
    - *Consumer key*
    - *Consumer secret*
    - *Access token*
    - *Access token secret*
    
Ya tienes todo lo que necesitas para crear tu *bot*.

## Ejemplo mínimo de *bot* con Python y `tweepy`.

El código que aparece a continuación contiene las líneas mínimas para crear un cliente de twitter y publicar un mensaje en twitter. 

Consejo: No lo ejecutes más de una vez o correrás el riesgo de que te baneen la cuenta.

In [ ]:
import tweepy
# añade las credenciales de tu aplicación de twitter como cadenas de texto
CONSUMER_KEY = 'CAMBIA ESTO'
CONSUMER_SECRET = 'CAMBIA ESTO' 
ACCESS_TOKEN = 'CAMBIA ESTO'
ACCESS_TOKEN_SECRET = 'CAMBIA ESTO'


# autentica las credenciales
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# crea un cliente de twiter
t = tweepy.API(auth)

# publica un saludo
t.update_status('¡Hola! Soy un bot, no me hagas caso.')

Hay que ser cuidadoso a la hora de enviar mensaje automatizados. Si queremos publicar más de un mensaje y queremos controlar el tiempo que transcurre entre un mensaje y otro, podemos utilizar la librería `time` de la siguiente manera:

In [ ]:
# añade esta librería al principio de tu código
import time

# algunas citas memorables de Yogi Berra (https://es.wikipedia.org/wiki/Yogi_Berra)
citas = '''The future ain't what it used to be.|
You can observe a lot by watching.|
It ain't over till it's over.|
It ain't the heat, it's the humility.|
We made too many wrong mistakes.|
I never said half the things I said.'''.split('|\n')

# iteramos sobre las citas y las publicamos de una en una 
for cita in citas:
    t.update_status(cita + ' #yogiberra')
    time.sleep(30) # envía el tweet cada 30 segundos

Otra funcionalidad típica que tienen los *bots* de twitter es enviar automáticamente mensajes a otros usuarios. Si queremos que nuestra aplicación busque entre todos los mensajes y responda al autor, podemos hacer lo siguiente:

In [ ]:
# buscamos mensajes que contengan la expresión "gaticos y monetes"
busqueda = t.search(q='gaticos y monetes')

# itero sobre estos mensajes
for mensaje in busqueda:
    # capturo el nombre de usuario
    usuario = mensaje.user.screen_name
    # compongo el mensaje de respuesta
    miRespuesta = '@%s ¡monetes!' % (usuario)
    # envío la respuesta
    mensaje = t.update_status(miRespuesta, mensaje.id)

Si queremos responder a menciones, es decir, si queremos que el *bot* responda automáticamente a mensajes dirigidos a él, podemos utilizar el siguiente código:

In [ ]:
# recupero las últimsa 5 menciones de mi usuario
menciones = t.mentions_timeline(count=5)

# ten en cuenta que:
# si tu cuenta de twitter es nueva y no tiene menciones, no funcionará
# si estás usando tu cuenta de todos los días, como hago yo, 
# enviarás mensajes a esas personas (o robotitos)

for mencion in menciones:
    # capturo el nombre de usuario que me manda el mensaje
    usuario = mencion.user.screen_name
    # compongo el mensaje de respuesta
    miRespuesta = '¡Hola, @%s! Soy un robotito. Este es un mensaje automático, no le hagas caso' % (usuario)
    # envío la respuesta
    mensaje = t.update_status(miRespuesta, mention.id)

Si queremos retuitear mensajes, podemos ejecutar lo siguiente: 


In [ ]:
# buscamos mensajes que contengan la expresión "viejóvenes"
busqueda = t.search(q='viejóvenes')

# para que no se descontrole, solo quiero retwitear los tres últimos mensajes
if len(busqueda) >= 3:
    for mensaje in busqueda[:3]:
        # para retwitear un mensaje, ejecuto el método retweet 
        # indicando el identificador único del mensaje en cuestión
        t.retweet(mensaje.id)
else:
    for mensaje in busqueda:
        t.retweet(mensaje.id)

Hasta aquí las funcionalidades básicas. Si quieres explorar más sobre las posibilidades de esta librería de twitter, revisa la documentación de [tweepy](https://tweepy.readthedocs.org/en/v3.5.0/).